This part is focused on exploring the data of the dataset. For this project we will use this [dataset of kaggle](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=movies_metadata.csv). We will use the table movies_metadata.

This analysis will enables me to clean the data (delete some useless collumns,...)

In [33]:
import pandas as pd
import numpy as np
import pandas_profiling

### Analysis

In [34]:
df = pd.read_csv('..\Data\movies_metadata.csv')
#pandas_profiling.ProfileReport(df)

C:\Users\Sacha\AppData\Local\Temp\ipykernel_14164\3842522359.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('..\Data\movies_metadata.csv')


### Cleaning
Some columns will be deleted and some not. This is mostly personal choice of what I want to display on my dashboard. I will delete the lines with nans, this is not compulsory but will be easier afterward to analyse and I think we have enough data to delete them.

The end goal is to create a tool in which the user enter a film name and then get +- 5 suggestions of similar movies (with duration, genre,...(possibly poster))

collums:

* __adult__: only 9 True, delete those lines and this columns
* __belongs_to_collection__: delete the collumn, we won't use it
* __budget__: mostly budget = 0, so drop the column
* __genres__: could be interesting to add to the dashboard -> we have to clean it -> create a list with the genre names only
* __homepages__: delete the column
* __id__: could be usefull after if we want to join others tables, keep it for now
* __imdb_id__: drop the duplicates(=same movie) + keep it for now
* __original_language__: I will work only on english movies (+delete the column)
* __original_title__: keep it
* __overview__: drop nan 
* __popularity__: drop it
* __poster_path__: keep it for now, and drop missing
* __production_companies__ : keep it + clean it
* __production_countries__: drop the column
* __release_date__: drop the missing values
* __revenue__: drop the column
* __runtime__: drop nans
* __spoken_languages__ : drop the column(require too much cleaning for the result)
* __status__ : keep only the Released and drop the column
* __tagline__: too much missing, drop the column
* __title__: drop it, we keep original title
* __video__: don't know what it is, only 93 True, keep only false and drop column (video available on the website?)
* __vote_average__: drop missing
* __vote_count__: drop the column

In [35]:
import ast

def extract_names(genres_list: str)-> str:
    """ 
    This function takes a str which can be converted in a list of dictionnary:                        henres_lit=[dic1,dic2,dic3]
    each of these dictionnaries have a key 'name':                                                    dic1={key1:..., name1:'...', key3:...,...}
    the function will return the values assosiated to these keys in a string (with , between names) : names = name1, name2, ...
    """
    #convert it into a list of dictionnary
    genres_list = ast.literal_eval(genres_list)

    #for each dictionnary, extract the name value    
    names = ""
    for genre_dict in genres_list:
        names+=(genre_dict['name'])+', '

    names=names[0:-2] #delete the last ', '
    return names

In [36]:
df = pd.read_csv('..\Data\movies_metadata.csv')

#drop nan for overview, poster_path
df = df.dropna(subset=['overview', 'poster_path', 'release_date', 'runtime', 'vote_average'])

#only non-adult movies
df = df[df['adult']=='False']
#only english movies
df = df[df['original_language'] == 'en']
#only released movies
df = df[df['status'] == 'Released']
#only false on video
df = df[df['video'] == False]
#print('adult', sum(df['video']!=False))

#add a column with the first character of the title (to select on the dashboard) +select only the ones with letter or numbers
df['first_title_char'] = df['original_title'].str[0]
df = df[df['first_title_char'].str.match(r'^[a-zA-Z0-9]')]

#drop the duplicates of imdb_id(=same movie)
df = df.drop_duplicates(subset=['imdb_id'])

#delete useless columns
df = df.drop(columns=['adult', 'belongs_to_collection', 'homepage', 'original_language', 'popularity', 
                      'production_countries', 'revenue', 'spoken_languages', 'status', 'tagline', 'title',
                       'video', 'vote_count', 'budget'])

#change imdb_id to imdb_link
df['imdb_id'] = 'https://www.imdb.com/title/' + df['imdb_id'] +'/'
df = df.rename(columns={'imdb_id':'imdb_link'})
#change the poster path to have the full link
df['poster_path'] = 'https://image.tmdb.org/t/p/w500' + df['poster_path']

#clean the 'genres' collumn
df['genres'] = df['genres'].apply(extract_names)

#clean production companies
df['production_companies'] = df['production_companies'].apply(extract_names)

#clean the runtime -> float is not necessary
df['runtime'] = df['runtime'].astype(int)

df.to_csv('..\data\movies_features.csv', index=False)
df.head()


C:\Users\Sacha\AppData\Local\Temp\ipykernel_14164\2516135720.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('..\Data\movies_metadata.csv')


,genres,id,imdb_link,original_title,overview,poster_path,production_companies,release_date,runtime,vote_average,first_title_char
0,"Animation, Comedy, Family",862,https://www.imdb.com/title/tt0114709/,Toy Story,"Led by Woody, Andy's toys live happily in his ...",https://image.tmdb.org/t/p/w500/rhIRbceoE9lR4v...,Pixar Animation Studios,1995-10-30,81,7.7,T
1,"Adventure, Fantasy, Family",8844,https://www.imdb.com/title/tt0113497/,Jumanji,When siblings Judy and Peter discover an encha...,https://image.tmdb.org/t/p/w500/vzmL6fP7aPKNKP...,"TriStar Pictures, Teitler Film, Interscope Com...",1995-12-15,104,6.9,J
2,"Romance, Comedy",15602,https://www.imdb.com/title/tt0113228/,Grumpier Old Men,A family wedding reignites the ancient feud be...,https://image.tmdb.org/t/p/w500/6ksm1sjKMFLbO7...,"Warner Bros., Lancaster Gate",1995-12-22,101,6.5,G
3,"Comedy, Drama, Romance",31357,https://www.imdb.com/title/tt0114885/,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",https://image.tmdb.org/t/p/w500/16XOMpEaLWkrcP...,Twentieth Century Fox Film Corporation,1995-12-22,127,6.1,W
4,Comedy,11862,https://www.imdb.com/title/tt0113041/,Father of the Bride Part II,Just when George Banks has recovered from his ...,https://image.tmdb.org/t/p/w500/e64sOI48hQXyru...,"Sandollar Productions, Touchstone Pictures",1995-02-10,106,5.7,F
